<a href="https://colab.research.google.com/github/JayRolla/Team-27-/blob/GIS-Data/Team27Internship(WIP).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We start by importing the neccessary packages, osmnx helps us access OpenStreetMap data in the python environment. Folium is a python package that will be used in visualisation of geo. Geopandas is used to work with geospatial data in python with the geopandas dataframe being utilised in the project.

In [1]:
! pip install osmnx
! pip install folium
! pip install geopandas
! pip install matplotlib 
! pip install mapclassify
! pip install ipyleaflet
import networkx as nx
import osmnx as ox
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
import folium
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
! pip install openrouteservice
from openrouteservice import client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


The next step is to import thefile containing the geometry for the desired location.

In [3]:
wards = gpd.read_file('/content/drive/MyDrive/Wards.geojson')


In [4]:
population = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/poulationincome - poulationincome.csv')

In [5]:
type_dict = {'WARD_NAME': 'int64'}
wards = wards.astype(type_dict)
population = population.astype(type_dict)

In [6]:
new_gpd = pd.merge(population, wards, on = 'WARD_NAME', how = 'right')

In [7]:
new_gpd = new_gpd[['OBJECTID', 'WARD_NAME', 'Total_Poplation', 'Area_(Sq Km)', 'Population_Density (People/Sq Km)', 'Average_Annual_Household_Income (Rands)', 'geometry']]

In [8]:
ward_geom = new_gpd['geometry']
ward_dens = new_gpd['Population_Density (People/Sq Km)']

In [9]:
Cen_gpd = new_gpd.copy()
Cen_gpd = gpd.GeoDataFrame(Cen_gpd)

In [10]:
cen = Cen_gpd['geometry']
centroids = cen.centroid
Cen_gpd['geometry'] = centroids
Cen_gpd = Cen_gpd[['OBJECTID', 'WARD_NAME', 'Total_Poplation', 'Area_(Sq Km)', 'Population_Density (People/Sq Km)', 'Average_Annual_Household_Income (Rands)', 'geometry']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [11]:
new_gpd = gpd.GeoDataFrame(new_gpd)

In [24]:
map2 = folium.Map(location=([-33.9289920, 18.4173960]), zoom_start=10)
bins = list(new_gpd['Population_Density (People/Sq Km)'].quantile([0, 0.25, 0.5, 0.75, 1]))
map2.add_child(folium.GeoJson('/content/Cen_gpd.geojson', name = 'Centroids'))
# Set up Choropleth map
folium.Choropleth(geo_data=new_gpd, data=new_gpd, columns=['OBJECTID',"Population_Density (People/Sq Km)"], 
                  Highlight= True, 
                  fill_color="YlGn",
                  fill_opacity=0.7,
                  line_opacity=.1,
                  legend_name="Population density",
).add_to(map2)
NIL = folium.features.GeoJson(data = new_gpd, tooltip=folium.features.GeoJsonTooltip(
    fields=['WARD_NAME','Population_Density (People/Sq Km)', 'Average_Annual_Household_Income (Rands)'],
        aliases=['Ward Name','Population Density (People/Sq Km)', 'Average Annual Household Income (Rands)'],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") 
    )
)
map2.add_child(NIL)

map2

The area of interest is Cape Town, since osmnx uses geocoding we save the variable place as 'City of Cape Town' so that the Nominatim API that supports osmnx can identify the area of interest

In [12]:
place = 'City of Cape Town'

The next step is to declare the tags. tags help the supporting API to search the specified area for a specific object, line or area. in this case the tag shop=supermarket helps us to search for supermarkets in the Cape Town region.

In [13]:
tags = {'shop':'supermarket'}

We use osmnx to perform the search based on the parameters defined above to retern a Geopandas dataframe that will contain information about all valid results.

In [14]:
shops = ox.geometries_from_place(place, tags=tags)
shops.head()

name         shop  \
element_type osmid                                     
node         26310714        Woolworths  supermarket   
             26313136        Pick n Pay  supermarket   
             205077126       Pick n Pay  supermarket   
             243992957  Woolworths Food  supermarket   
             243992967         Checkers  supermarket   

                                          geometry       brand brand:wikidata  \
element_type osmid                                                              
node         26310714   POINT (18.51046 -33.93310)         NaN            NaN   
             26313136   POINT (18.51101 -33.93421)  Pick n Pay       Q7190735   
             205077126  POINT (18.47041 -33.96067)  Pick n Pay       Q7190735   
             243992957  POINT (18.47049 -33.95994)         NaN            NaN   
             243992967  POINT (18.47012 -33.96239)    Checkers       Q5089126   

                                   brand:wikipedia  \
element_type osmid                                   
node         26310714                          NaN   
             26313136         en:Pick n Pay Stores   
             205077126        en:Pick n Pay Stores   
             243992957  en:Woolworths Supermarkets   
             243992967                         NaN   

                                                            opening_hours  \
element_type osmid                                                          
node         26310714                                                 NaN   
             26313136                                                 NaN   
             205077126                                  Mo-Su 08:00-21:00   
             243992957                                                NaN   
             243992967  Mo-Fr 08:00-19:00; Sa 08:00-17:00; Su 09:00-16:00   

                       wheelchair addr:postcode addr:street  ... diet:halal  \
element_type osmid                                           ...              
node         26310714         NaN           NaN         NaN  ...        NaN   
             26313136         NaN           NaN         NaN  ...        NaN   
             205077126        yes           NaN         NaN  ...        NaN   
             243992957    limited           NaN         NaN  ...        NaN   
             243992967    limited           NaN         NaN  ...        NaN   

                       diet:kosher name:af organic ways type designation  \
element_type osmid                                                         
node         26310714          NaN     NaN     NaN  NaN  NaN         NaN   
             26313136          NaN     NaN     NaN  NaN  NaN         NaN   
             205077126         NaN     NaN     NaN  NaN  NaN         NaN   
             243992957         NaN     NaN     NaN  NaN  NaN         NaN   
             243992967         NaN     NaN     NaN  NaN  NaN         NaN   

                       alt_name:en alt_name:en2 addr:housename  
element_type osmid                                              
node         26310714          NaN          NaN            NaN  
             26313136          NaN          NaN            NaN  
             205077126         NaN          NaN            NaN  
             243992957         NaN          NaN            NaN  
             243992967         NaN          NaN            NaN  

[5 rows x 57 columns]

We can see that the dataframe contains a lot of data, we can clean it by subsetting the data and then removing the null values from the data.

In [15]:
shops = shops[['name', 'shop', 'geometry']]
shops = shops.dropna()

We can use the `explore()` method available from geopandas and uses folium and matplotlib to create an interactive map that includes the information in the dataframe and over lays it onto a OpenStreetMap base map.

In [16]:
center = (-33.9289920, 18.4173960)
m = Map(center=center, basemap=basemaps.CartoDB.Positron, zoom=10)

geo_data = GeoData(geo_dataframe = shops, name = 'Shops')

m.add(geo_data)
m.add(LayersControl())
m

Map(center=[-33.928992, 18.417396], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

In [17]:
Cen_gpd.head()

Cen_gpd.to_file("Cen_gpd.geojson", driver='GeoJSON')

In [18]:
api_key = '5b3ce3597851110001cf62480ac62be4cc3747a890840c0cfde8bd1d'  # Provide your personal API key
ors = client.Client(key=api_key)
# Set up folium map
map1 = folium.Map(location=([-33.9289920, 18.4173960]), zoom_start=10)
map1.add_child(folium.ClickForMarker())
map1

In [19]:
api_key = '5b3ce3597851110001cf62480ac62be4cc3747a890840c0cfde8bd1d'  # Provide your personal API key
ors = client.Client(key=api_key)
# Set up folium map
map1 = folium.Map(location=([-33.9249, 18.4241]), zoom_start=10)

# Set up the apartment dictionary with real coordinates
apartments = {'first': {'location': [18.4193, -33.9287]},
              'second': {'location': [18.4919, -33.5628]},
              'third': {'location': [18.4286, -34.1709]}
              }

# Request of isochrones with 20 minute walking.
params_iso = {'profile': 'foot-walking',
              'range': [900],  
              'attributes': ['total_pop']  # Get population count for isochrones
              }

for name, apt in apartments.items():
    params_iso['locations'] = [apt['location']]  # Add apartment coords to request parameters
    apt['iso'] = ors.isochrones(**params_iso)  # Perform isochrone request
    folium.features.GeoJson(apt['iso']).add_to(map1)  # Add GeoJson to map

    folium.map.Marker(list(reversed(apt['location'])),  # reverse coords due to weird folium lat/lon syntax
                      icon=folium.Icon(color='lightgray',
                                       icon_color='#cc0000',
                                       icon='home',
                                       draggable = True,
                                       prefix='fa',
                                       ),
                      popup=name,
                      ).add_to(map1)  # Add apartment locations to map

map1


In [21]:
# Common request parameters
params_poi = {'request': 'pois',
              'sortby': 'distance'}

# POI categories according to
# https://giscience.github.io/openrouteservice/documentation/Places.html
categories_poi = {'Taxi stops': [607],
                  'Parking Available': [601],
                  'Competitors': [518]}

for name, apt in apartments.items():
    apt['categories'] = dict()  # Store in pois dict for easier retrieval
    params_poi['geojson'] = apt['iso']['features'][0]['geometry']
    print("\n{} location".format(name))

    for typ, category in categories_poi.items():
        params_poi['filter_category_ids'] = category
        apt['categories'][typ] = dict()
        apt['categories'][typ]['geojson'] = ors.places(**params_poi)['features']  # Actual POI request
        print(f"\t{typ}: {len(apt['categories'][typ]['geojson'])}")


first location
	Taxi stops: 70
	Parking Available: 100
	Competitors: 11

second location
	Taxi stops: 24
	Parking Available: 6
	Competitors: 2

third location
	Taxi stops: 3
	Parking Available: 2
	Competitors: 1


In [25]:
map3 = folium.Map(location=([-33.9289920, 18.4173960]), zoom_start=10)
map3.add_child(folium.Circle())
map3

ValueError: ignored